# Using Python to Access the Geopolitical Forecasting Challenge API
## Introduction

Participation in the Geopolitical Forecasting (GF) Challenge is conducted through an API provided by Cultivate Labs. This notebook briefly demonstrates some ways to interact with the API through Python. These examples assume that you are familiar with Python 2 or 3, and have installed the [Requests](http://docs.python-requests.org/en/master/) library (which is included in distributions such as Anaconda). (NOTE: You may use these examples as part of your GF Challenge solution, but it is important to note that this code is meant primarily as a reference, and should not be assumed to be bug-free, or particularly efficient. We make no guarantees or warranties that this code will correctly retrieve or submit data to the API -- you are responsible for verifying that your forecasts are submitted correctly).

Prior to participating in the GF Challenge, you must register for the GF Challenge at [HeroX](https://www.herox.com/IARPAGFChallenge), and then register on the Cultivate platform using the URLs provided. Upon registering on the Cultivate platform, you will be able to generate an API key for the staging (test) instance, and production (competition) instance of the API. This API key is unique to you, and you should ensure that it is not shared with others.  This API key is used to identify and authenticate your requests and submissions to the API.

You will find complete API documentation on the Cultivate Labs site once you receive your API key. This notebook does not provide an exhaustive overview of the entire API. Instead, it is designed to highlight some key considerations for implementing a GF Challenge API client. It is not a substitute for a thorough understanding of the API documentation. This document describes some concepts related to the challenge, however, it is not an authoritative source of challenge rules. You are responsible for reviewing and understanding the official GF Challenge Rules document available on the HeroX site.

All code in this document is provided using the [CC0 1.0 Universal (CC0 1.0) Public Domain Dedication](https://creativecommons.org/publicdomain/zero/1.0/).

## General Details

### Using your API Key

All calls to the API must include your API key in the request headers in the following form:

`headers['Authorization'] = 'Bearer ' + secret_token`

So, one can submit a GET request to the API this way:

In [ ]:
import requests

secret_token = '<API_KEY>' # replace with your API token
server = '<SERVER_NAME>'  # of the form 'https://api.[domain].com'
url = server + '/api/v1/questions' # The endpoint to retrieve questions
headers = {'Authorization':'Bearer ' + secret_token}
params = {} # More to come on this in a moment

result = requests.get(url, headers=headers, params=params) 

if result.ok:
    j = result.json() # This will be the content you are interested in.
else:
    print('PROBLEM:', result.status_code, result.text)

When POSTing forecasts, things look pretty much the same, except it would be formatted with the `.post()` method, and the parameters would be submitted as `json`:

`result = requests.post(url, headers=headers, json=params)`

### Passing parameters to the API

Typically, you'll want to provide specific parameters when making your API calls, (e.g., asking for human forecasts made against a specific forecasting question, requesting information that has been updated since you last checked).  To do that, you will pass a set of parameters in python dictionary form.  The dictionary structure will be identical to the examples provided in the Cultivate API documentation.

As an example, you may want to receive all human forecasts made against question number 5 since March 10, 2018. You would set your params dictionary as:

`params = {'question_id':5, 'created_after':'2018-03-10T00:00:00.000Z'}`

The full set of required and optional input parameters are listed in the Cultivate API documentation.

### Recieving Responses

When a GET or POST request is successfully executed, you will receive a JSON formatted response which can be accessed as the response's `.json()` object.

#### Paging

All paginated endpoints will also include 2 pagination-related response headers: `X-Total-Page-Count` and `X-Total-Record-Count`. `X-Total-Page-Count` contains the total number of pages available for your request, while `X-Total-Record-Count` contains the total number of records that will be included across all of those pages.

You can access these through the response's `headers` dictionary:

```
result = requests.get(url, headers=headers, params=params)

if result.ok:
    totalPages = int(resp.headers.get('X-Total-Page-Count',0))
```

By default, results are returned for the first page (page 0), updating the `params` dictionary to include `params['page'] = 1` would get the next page.

## Primary API Endpoints

### Retrieving Questions

Individual Forecasting Problems (IFPs) are questions about future events that solvers forecast against, and can be retrieved from the `questions` API endpoint. Each IFP includes, among other fields, an `id` a `description`, a set of `answers`, and a collection of `metadata` as well as starting and ending dates. Questions can be retrieved using a GET request, optionally specifying the `status` (active, closed, all) or limiting the dates questions were updated or created. The `answers` are a list of mutually exclusive, and collectively exhaustive options describing possible IFP outcomes. Forecasts must specify probabilities for each possible outcome that sum to 1.0 (except for binary questions (e.g., yes/no) for which only one option is presented, with the other option being calculated as 1 - Option A).

The `metadata` that is returned includes `Domain`, `Topic`, and location information. These fields come into play in the interim prizes and the Domain/Region pair prizes.  Refer to the GF Challenge Rules document and the API documentation to ensure that you accurately retrieve and handle the relevant fields. 

Each IFP also includes a list of `clarifications`. This will be populated if there is additional guidance regarding the IFP issued. This can include situations where terms are further defined, or sources of resolution are changed. You should regularly check for updates to this field.

### Retrieving Human Forecasts

As part of the GF Challenge, solvers will have access to forecasts made by a crowd of human forecasters. These forecasts will be made available in two forms: individual and aggregate.  The individual forecasts are made available through the `prediction_sets` API end point, while the aggregate forecasts are provided through the `consensus_histories` end point. The consensus is calculated using an aggregation algorithm called logit. The logit aggregation method is an extremizing method that uses a weighted geometric mean to aggregate forecasts. Forecaster weights are calculated based on 3 factors: historical accuracy, the frequency with which the forecaster updates his or her forecasts, and whether the forecaster completed a training course. The `consensus_histories` data also serve as the baseline against which Solvers will be compared. More details on the baseline and scoring can be found in the official GF Challenge Rules.

Individual forecasts contain the probability forecasts, including a `question_id` that maps to the `id` in from the `questions` endpoint and `membership_guid` which uniquely identifies the human forecaster. Each forecast will include a list of `predictions` reflecting the probabilities that person assigned to each possible answer. The `forecasted_probability` for each answer represents that person's beliefs for each answer.

Consensus aggregations contain a `question_id` and `answer_id` pair that identify a particular answer option. The `normalized_value` for a particular answer reflects the score such that all answers to a particular question will sum to 1.0. This consensus is updated any time a human forecaster makes a new forecast against an IFP. The `consensus_histories` API end point contains a list of these updated consensus scores. The most recent consensus for a particular question reflects the current crowd consensus at that time. NOTE: With the exception of the first time you query this endpoint, you should NOT retrieve `consensus_histories` without specifying a `created_after` parameter.  

### Submitting Forecasts

Forecast submission is done through an HTTP POST. Your forecast must include the `question_id`, an `external_predictor_attributes.method_name`, and `external_predictions_attributes`: a list of dictionaries each containing an `answer_id` and `value` for each of the forecast question's possible alternatives. The sum of the values must equal 1.0. 

Each GF Challenge solver is allocated 25 methodological "slots." These slots can be used to represent different strategies for weighting data sources, different algorithms, etc. The `method_name` parameter is used to identify which slot a forecast should be associated with. `method_name` can be up to 50 characters, and will be held constant throughout the challenge (i.e., you cannot add a 26th method). You will be scored on a per `method_name` basis, with only your best performing approach being considered for each prize category. For more details, review the GF Challenge rules.

A forecast submission can look like:

In [ ]:
params = {"external_prediction_set": {
        "question_id": 123,
        "external_predictor_attributes": {
            "method_name": "red"
            },
        "external_predictions_attributes": [
            {"value": 0.6, "answer_id": 431},
            {"value": 0.35, "answer_id": 432},
            {"value": 0.05, "answer_id": 433}
            ]
          }}

A successful submission of a forecast to the submission endpoint will return a json summary of the submission, including the time of submission. A failure (e.g., incorrect `answer_id` or `value`s that don't total to 1.0, or attempting to create a 26th `method_name`) will result in json describing the error. It is advisable to inspect the resulting json to ensure it reflects the intended forecast.

## Putting it All Together

We can implement API access into a single Python class so we can make GET and POST requests in a consistent fashion.  Below, we define a `GfcApi` class that allows us to specify a server and API token one time and access all the API endpoints.

In [2]:
import requests
import time
import datetime

from pprint import pprint #This is just to make things look pretty...

#Make this python 2 and 3 compliant
from __future__ import print_function

class GfcApi(object):
    """
        An example class for interacting with the IARPA Geopolitical Forecasting Challenge
        API.  Note that this code is for reference purposes, no warranties are expressed
        or implied.  
    """
    def __init__(self,token,server,proxy=None,verbose=False):
        """
            Create an instance of an API client. This assumes you have an OAuth token.
            
            Arguments
            
            REQUIRED
            token - <string> - The secret API token assigned when registering on the 
                               Cultivate platform
            
            server - <string> - The beginning of the server url in the form:
                                https://api.XXXXXXX.com.  This is described in the
                                Cultivate API documentation
            
            OPTIONAL
            proxy - <dictionary> - If you are behind a proxy server, you can specify the details
                                   in the form: 
                                       proxy = {'http': http_proxy,
                                                'https': https_proxy,
                                                'ftp': ftp_proxy}
                                   where an individual entry might be [ip address:port]. See the
                                   requests library documentation for more details.
                Default: None
                
            verbose - <boolean> - If true, we print GET and POST request URLs and params
                Default: False
            
        """
        
        self.token = token
        self.server = server
        self.proxy = proxy
        self.verbose = verbose
        
        self.sess = requests.session()
        self.rate_limit_delay = 1 #seconds between subsequent API calls
        self.last_call_time = 0.0 
        self.set_urls()
    
    def set_urls(self):
        if not self.server.endswith('/'):
            self.server += '/'
    
        self.api_base = self.server + 'api/v1/'
    
        self.consensus_histories_url = self.server + 'aggregation/api/v1/control/consensus_histories'
        self.external_prediction_sets_url = self.api_base + 'external_prediction_sets'
        self.prediction_sets_url = self.api_base + 'control/prediction_sets'
        self.questions_url = self.api_base + 'questions'

    def get_questions(self, status=None, created_before=None, created_after=None,
                  sort='published_at', updated_before=None, updated_after=None):
        """
            This function retrieves Individual Forecasting Problems (IFPs).

            Optional Inputs:
            status - <string> - IFP status
                    Possible Values:
                        'active' - only return questions that are currently open for forecasting
                        'closed' - return all resolved or otherwised closed questions
                        'all'    - return all active and closed questions
                    Default Value:
                        'active'

            created_before - <datetime> - returns only questions created before this time

            created_after - <datetime> - returns only questions created after this time
            
            sort - <string> - Sort order of returned questions
                    Possible Values:
                        'published_at'
                        'ends_at'
                        'resolved_at'
                        'prediction_sets_count'
                    Default Value:
                        'published_at'
            
            updated_before - <datetime> - returns only questions updated before this time
            
            updated_after - <datetime> - returns only questions updated after this time
                    
             Output:
            JSON representation of a list of Individual Forecasting Problems
        """
        
        url = self.questions_url
        section = 'questions'
        params={}
        
        if created_before:
            params['created_before'] = created_before.isoformat()
        if created_after:
            params['created_after'] = created_after.isoformat()
        if created_before:
            params['updated_before'] = updated_before.isoformat()
        if created_after:
            params['updated_after'] = updated_after.isoformat()
        if status:
            params['status'] = status
        if sort:
            params['sort'] = sort  
        
        return self._get_pages(url=url,section=section,params=params)
    
    def get_human_forecasts(self, question_id=None, created_before=None, created_after=None,
                           updated_before=None, updated_after=None):

        """
            This function retrieves the stream of human forecasts against IFPs.

            Optional Inputs:
            question_id - <integer> - returns predictions for a single question
                    Default Value:
                        None

            created_before - <datetime> - returns only predictions created before this time

            created_after - <datetime> - returns only predictions created after this time
            
            updated_before - <datetime> - returns only predictions updated before this time
            
            updated_after - <datetime> - returns only predictions updated after this time
                    
             Output:
            JSON representation of a list of human forecasts
        """
        
        url = self.prediction_sets_url
        section = 'prediction_sets'
        params={}
        
        if created_before:
            params['created_before'] = created_before.isoformat()
        if created_after:
            params['created_after'] = created_after.isoformat()
        if updated_before:
            params['updated_before'] = updated_before.isoformat()
        if updated_after:
            params['updated_after'] = updated_after.isoformat()
        if question_id:
            params['question_id'] = question_id
        
        return self._get_pages(url=url,section=section,params=params)        
    
    def get_consensus_histories(self, question_id=None, created_before=None, created_after=None,
                           updated_before=None, updated_after=None):

        """
            This function retrieves the consensus of human forecasts against IFPs.

            NOTE: You need to include some date constraints after your first use of this API. 
            Always utilize the created_after parameter to pull only those records that have 
            been created since you last accessed the API. Do not attempt to pull every 
            record/page of the history.

            Optional Inputs:
            
            question_id - <integer> - returns only predictions made about a specific IFP
                Default Value
                    None

            created_before - <datetime> - returns only predictions created before this time

            created_after - <datetime> - returns only predictions created after this time
            
            updated_before - <datetime> - returns only predictions updated before this time
            
            updated_after - <datetime> - returns only predictions updated after this time
                    
             Output:
            JSON representation of a list of human forecasts
        """
        
        if (not created_before) and (not created_after) and (not updated_before) and (not updated_after):
            print("After your first query, use a date constraint (created_before/after or",\
                  "updated_before/after) to get consensus history. Old values won't change")
        
        url = self.consensus_histories_url
        section = 'consensus_histories'
        params={}
        
        if question_id:
            params['question_id'] = str(question_id)
        if created_before:
            params['created_before'] = created_before.isoformat()
        if created_after:
            params['created_after'] = created_after.isoformat()
        if updated_before:
            params['updated_before'] = updated_before.isoformat()
        if updated_after:
            params['updated_after'] = updated_after.isoformat()
        
        return self._get_pages(url=url,section=section,params=params)   
    
    def submit_forecast(self,question_id,method_name,predictions):
        """
            Submit probabilistic forecasts against a question.
            
            Required Parameters
            
            question_id - <integer> - The question_id of the IFP being forecast against
            
            method_name - <string> - The name of one of your 25 forecasting methods. Up to 50 chars
                             NOTE: This is used to track and score your forecasting methods. You
                             are responsible for keeping track of your named methods. Using a new
                             method_name will automatically add a new method - unless you have
                             already created 25 methods. In that case, you'll get an error message
                             in the response.
                             
            predictions - <list> - A list of Dictionaries in the form .
                                                   {'answer_id': <Integer>, 'value': <Decimal>}
                            
                          If the question is binary (exactly two possible answers), you only submit a
                          prediction for one possible answer, with the other being equal to 1 minus
                          your prediction for option A.
                          
                          NOTE: The set of values in the forecast must equal exactly 1.0 or you will 
                          receive an error message in the response.
            
     RESPONSE
     The json response will either summarize your forecast to this question, or it will contain an 
     error message indicating why it wasn't accepted.  You are responsible for recieving and reviewing
     the response to ensure that your forecast was accepted, and reflects your intentions.  You can 
     resubmit forecasts to a particular IFP repeatedly over the course of a forecast day, and each
     new submission will replace older submissions for scoring purposes. Review the GF Challenge
     Rules for details on forecast submission and scoring.
        """
    
        url = self.external_prediction_sets_url
        
        params={'external_prediction_set':{'question_id':question_id,
                                          'external_predictor_attributes':
                                           {'method_name':method_name},
                                           'external_predictions_attributes':predictions}
                }
        
        return self._post(url,params)
    
    def _forecast_template(self,ifp):
        """
            A tiny little helper function to create the basis for the predictions parameter in
            the submit_forecast function.  You pass an IFP from the questions API into this 
            function and receive a list of 'answer_id' and 'value' dictionaries that are
            needed to submit a forecast.  
            
            NOTE: This uses the existing 'probability' value from the questions API which should be
            replaced with your own forecast values.
        """
        
        output = [{'answer_id':a['id'],'value':a['probability']} for a in ifp['answers']]
        return output
    
    def _get_pages(self,url,params,section):
        
        """
            This function uses _get to make authenticated calls to the
            relevant API endpoints with the user-provided parameters.
            
            This function handles paging through results, and returns only the list from
            the resulting json result(s).
            
            The 'url' and 'params' describe the API query, the 'section' is the key in the
            returned json that contains the list of query results (e.g., 'questions').
        """
        if self.verbose:
            print('Get Pages for {}'.format(url))
            print(params)
        page = 0
        maxPage = 1
        
        all_results = []
        this_batch = []
        while page < maxPage: 
            
            params['page']=page
            resp = self._get(url=url,params=params)
            maxPage = int(resp.headers.get('X-Total-Page-Count',0))
            try:
                results=resp.json()
            except:
                results=None
            if isinstance(results,(list,dict)):
                if 'errors' in results:
                    print(results['errors'])
                    return results
                
                this_batch = results[section]
                all_results.extend(this_batch)

                page+=1
            else:
                if self.verbose:
                    print("PROBLEM")
                return results

        return all_results                
        
    def _get(self,url,params):
        """
            A helper function that handles authentication and rate limiting.
            
            Given a URL and a set of parameters, this function calls the Cultivate API
            and returns the json response.
        """
        
        while time.time() < self.last_call_time + self.rate_limit_delay:
            if self.verbose:
                print("{}: Sleeping".format(time.ctime()))
            time.sleep(1)
        
        headers={'Authorization':'Bearer ' + self.token} #This is needed to authenticate

        if self.verbose:
            print("{}: GETTING {}".format(time.ctime(),url))
            safeHeaders = {k:v for k,v in headers.items() if k!='Authorization'}
            safeHeaders['Authorization']="Bearer <shhhhhh it's a secret>"
            print("\tHeaders: {}".format(safeHeaders))
            print("\tArgs: {}".format(params))
        resp = self.sess.get(url, headers=headers, params=params, proxies=self.proxy)
                                                                                         
        self.last_call_time = time.time()
        return resp
    
    def _post(self,url,params):
        """
            A helper function that handles authentication.
            
            Given a URL and a set of parameters, this function submits a POST to the 
            Cultivate API and returns the json response.
            
            Output
            JSON response describing the forecast or indicating an error.

        """

        while time.time() < self.last_call_time + self.rate_limit_delay:
            if self.verbose:
                print("{}: Sleeping".format(time.ctime()))
            time.sleep(1)
        
        headers={'Authorization':'Bearer ' + self.token} #This is needed to authenticate

        if self.verbose:
            print("{}: POSTING {}".format(time.ctime(),url))
            safeHeaders = {k:v for k,v in headers.items() if k!='Authorization'}
            safeHeaders['Authorization']="Bearer <shhhhhh it's a secret>"
            print("\tHeaders: {}".format(safeHeaders))
            print("\tArgs: {}".format(params))
        resp = self.sess.post(url, headers=headers, json=params, proxies=self.proxy) 
                                                                                         
        self.last_call_time = time.time()
        
        return resp.json()

We can invoke this class by specifying our secret_token and server.

One strategy for token management is to create a dictionary of server instances with the server address and API token like:

```
secrets = {'staging':{'key':'<API KEY>','server':'https://api.<STAGING DOMAIN>.com'},
          'production':{'key':'<API KEY>','server':'https://api.<PRODUCTION DOMAIN>.com'}}
```
We can create an instance of the `GfcApi` class thusly:

In [3]:
instance='staging'
gf=GfcApi(secrets[instance]['key'],secrets[instance]['server'],verbose=False)

Once we create an instance of the `GfcApi` class, we retrieve Individual Forecasting Problems (IFPs). We could limit our queries of IFPs based on date of creation or update (useful for finding clarifications).  We can also limit our query to active (or closed) questions.

In [4]:
ifps=gf.get_questions()
print("We've downloaded {} IFPs\n".format(len(ifps)))

for ifp in ifps:
    print("IFP {}: {}".format(ifp['id'],ifp['name']))
    print("Description: {}".format(ifp['description']))
    print("Starts: {}, Ends: {}".format(ifp['starts_at'],ifp['ends_at']))
    print("Options:")
    for answer in ifp['answers']:
        print(' ({}) {}'.format(answer['id'],answer['name']))
        
    if ifp['clarifications']:
        print('Clarifications:')
        print(ifp['clarifications'])
    print("")    


We've downloaded 4 IFPs

IFP 325: Single Answer question test
Description: 
Starts: 2018-02-15T14:57:00.000Z, Ends: 2018-03-15T14:57:00.000Z
Options:
 (685) Ice

IFP 256: test ordinal question
Description: this is a description
Starts: 2018-02-10T19:01:01.000Z, Ends: 2018-03-10T19:01:01.000Z
Options:
 (511) answer a
 (512) answer b
 (513) answer c

IFP 211: test binary question
Description: 
Starts: 2018-01-21T18:55:38.000Z, Ends: 2018-06-01T17:55:38.000Z
Options:
 (446) Yes

IFP 221: test multinomial
Description: 
Starts: 2018-02-05T16:45:01.000Z, Ends: 2018-08-04T15:44:34.000Z
Options:
 (456) A
 (457) B
 (458) C



We can retrieve human forecasts. If we'd like, we can limit them to a particular `question_id`, and can constrain the creation or update dates. 

In [5]:
preds=gf.get_human_forecasts()
if 'errors' in preds:
    print("We ran into a problem:")
    print(preds)
else:
    print("Retrieved {} human forecasts".format(len(preds)))

Retrieved 5 human forecasts


Let's look at an item in the human forecast stream. The `question_id` links us to the `get_questions()` results. the `membership_guid` is the unique identifier for a human forecaster, and will remain consistent throughout the GF Challenge.

Each item in the `predictions` list includes the `answer_id` for that alternative, which aligns to the `get_questions()` output, and a `forecasted_probability` which indicates the human forecaster's submitted probability for that alternative.

In [6]:
preds[1]

{'comment_id': 15,
 'created_at': '2018-02-15T19:44:14.707Z',
 'discover_question_id': 40,
 'id': 4,
 'membership_guid': 'cc4c30df5539f534690f0a36209738ce78a04180',
 'predictions': [{'answer_id': 685,
   'answer_name': 'Ice',
   'confidence_level': None,
   'created_at': '2018-02-15T19:44:14.724Z',
   'filled_at': '2018-02-15T19:44:14.724Z',
   'final_probability': 0.15,
   'forecasted_probability': 0.15,
   'id': 10,
   'made_after_correctness_known': False,
   'membership_guid': 'cc4c30df5539f534690f0a36209738ce78a04180',
   'refunded_at': None,
   'starting_probability': 0.5,
   'type': 'Forecast::Prediction',
   'updated_at': '2018-02-15T21:45:08.420Z'}],
 'question_id': 325,
 'question_name': 'Single Answer question test',
 'rationale': 'test rationale',
 'type': 'Forecast::OpinionPoolPredictionSet',
 'updated_at': '2018-02-15T21:45:08.413Z'}

We can retrieve the baseline consensus forecasts using `get_consensus_histories()`. As described in the API documentation, and above, after your first call to this API endpoint, you should constrain your requests using something like `created_after` while storing tracking older values locally. Note that we're using `datetime.datetime()` objects to specify the `created` and `updated` parameters. You can limit this request by `question_id` if desired.

In [7]:
cons = gf.get_consensus_histories(created_after=datetime.datetime(2018,1,1,0,0,0),
                                  updated_before=datetime.datetime(2018,2,11)) 
print("retrieved {} consensus scores".format(len(cons)))

retrieved 9 consensus scores


Let's look at these results. Note that each item in the list represents a single answer -- unlike an item in the `get_human_forecasts()` results where each entry represents the predictions for each possible answer for a single IFP.

The `normalized_value` scores for all the answers to a single IFP for a specific `consensus_at` time will add up to 1.0.

In [8]:
cons

[{'answer_id': 511,
  'consensus_at': '2018-02-10T23:06:52.065Z',
  'created_at': '2018-02-10T23:06:52.674Z',
  'decay_args': {'percent': 0.468},
  'decay_method': 'Aggregation::Decay::PercentRecent',
  'id': 15,
  'method_name': '1-WeightedLogit-PercentRecent',
  'normalized_value': 0.45000005,
  'prediction_set_id': 3,
  'question_id': 256,
  'strategy': 'Aggregation::Strategies::Logit',
  'updated_at': '2018-02-10T23:06:52.674Z',
  'value': 1.0,
  'weighting_settings': {'count_of_closed_questions_answered_requirement': 35,
   'enabled': True,
   'minimum_closed_questions_to_enable_accuracy_weighting': 10,
   'percentage_of_closed_questions_answered_requirement': 0.5}},
 {'answer_id': 512,
  'consensus_at': '2018-02-10T23:06:52.065Z',
  'created_at': '2018-02-10T23:06:52.650Z',
  'decay_args': {'percent': 0.468},
  'decay_method': 'Aggregation::Decay::PercentRecent',
  'id': 16,
  'method_name': '1-WeightedLogit-PercentRecent',
  'normalized_value': 0.14999986,
  'prediction_set_id':

Creating and submitting a forecast
---

The `submit_forecast()` function is used to submit probabilistic forecasts against IFPs.  Each forecast must include the `question_id`, a `method_name`, and `predictions`, a list of question_id and value dictionaries that reflect your beliefs about how the IFP will resolve. The sum of all the values must equal 1.0, unless the IFP is a binary question (having only two possible outcomes), in which case, only the forecast for Option A is submitted, with Option B being inferred to be 1 minus the value of Option A.

We can make this process a bit easier by taking advantage of the `_forecast_template()` function to create a starting list of forecasts.

In [9]:
myIFP = ifps[1]
id=myIFP['id']

method="Roll a d20"

forecasts = gf._forecast_template(myIFP)

forecasts[0]['value'] = 0.5  #Set our probability for this option
forecasts[1]['value'] = 0.1
forecasts[2]['value'] = .4

print('Forecasting IFP {}, using the "{}" method:'.format(id,method))
print(forecasts)

result = gf.submit_forecast(id,method,forecasts)
print("\n")
pprint(result)


Forecasting IFP 256, using the "Roll a d20" method:
[{'value': 0.5, 'answer_id': 511}, {'value': 0.1, 'answer_id': 512}, {'value': 0.4, 'answer_id': 513}]


{'created_at': '2018-02-22T20:30:30.643Z',
 'discover_question_id': 39,
 'external_predictions': [{'answer_id': 511,
                           'created_at': '2018-02-22T20:30:30.653Z',
                           'discover_answer_id': 78,
                           'forecast_at': '2018-02-22T20:30:30Z',
                           'id': 108,
                           'membership_id': 26,
                           'method_name': 'Roll a d20',
                           'question_id': 256,
                           'site_id': 2,
                           'updated_at': '2018-02-22T20:30:30.653Z',
                           'value': 0.5},
                          {'answer_id': 512,
                           'created_at': '2018-02-22T20:30:30.658Z',
                           'discover_answer_id': 79,
                           'fo

Let's take a look at what happens when your answers sum to something other than 1.0

In [10]:
myIFP = ifps[1]
id=myIFP['id']

method="Roll a d20"

forecasts = gf._forecast_template(myIFP)

forecasts[0]['value'] = 0.6  #This is different than the previous entry, and will push our total to 1.1
forecasts[1]['value'] = 0.1
forecasts[2]['value'] = .4

print('Forecasting IFP {}, using the "{}" method:'.format(id,method))
print(forecasts)

result = gf.submit_forecast(id,method,forecasts)
print("\n")
pprint(result)


Forecasting IFP 256, using the "Roll a d20" method:
[{'value': 0.6, 'answer_id': 511}, {'value': 0.1, 'answer_id': 512}, {'value': 0.4, 'answer_id': 513}]


{'errors': {'predictions': ['must add up to 100%']}}


## Closing Thoughts
Your solution to the GF challenge will need to interact, at the very least with the `questions` and `external_prediction_sets` end points. You will need to make authenticated calls to GET and POST to these end points. You may use any of the concepts or code in this document to help you accomplish these tasks, but please note that you are responsible for ensuring that you are correctly retrieving and submitting information to the API.

Note that you should be on the lookout for errors that are returned either because of connectivity issues (Requests library) or as returned json content from the Cultivate API.  The examples in this document don't include comprehensive error checking or handling.